**Objectif du projet** : Afficher les trajets possibles en Bus/TER en partant d'une ville précise en prenant en compte le prix et les correspondances.

- Créer une plateforme interagissable avec à gauche un rendu des différentes villes avec les trajet détaillé pour s'y rendre de la ville de départ et à droite une vue carte des villes disponibles avec les critères pris en compte.

- Prendre en compte les différentes plateforme de transports (SNCF, FlixBus, Blablabus, OuiGo, etc...)

- Critère de prix, de transport, d'horaires, de jours, de ville

In [1]:
# Import des modules nécessaires

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import folium

#Données lues via Pandas de la route allant sur le colab de mon Drive
data = 'routes.txt'
FlixbusData = pd.read_csv(data)

Affichage des données chargées si besoin

In [2]:
FlixbusData

,agency_id,route_id,route_short_name,route_long_name,route_type,route_desc,route_url,route_color,route_text_color,bikes_allowed,route_sort_order
0,FLIXBUS-eu,N862,FlixBus N862,Prague - Pula,3,NaN,NaN,73D700,FFFFFF,NaN,NaN
1,FLIXBUS-eu,858,FlixBus 858,Prague - Chomutov - Most,3,NaN,NaN,73D700,FFFFFF,NaN,NaN
2,FLIXBUS-eu,507,FlixBus 507,Geneva - Milan - Bologna,3,NaN,NaN,73D700,FFFFFF,NaN,NaN
3,FLIXBUS-eu,563,FlixBus 563,Rome - Sorrento,3,NaN,NaN,73D700,FFFFFF,NaN,NaN
4,FLIXBUS-eu,455,FlixBus 455,Milan - Innsbruck - Munich,3,NaN,NaN,73D700,FFFFFF,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1117,FLIXBUS-eu,N31,FlixBus N31,Berlin - Rotterdam - Brussels,3,NaN,NaN,73D700,FFFFFF,NaN,NaN
1118,FLIXBUS-eu,UK045,FlixBus UK045,London - Plymouth,3,NaN,NaN,73D700,FFFFFF,NaN,NaN
1119,FLIXBUS-eu,N789,FlixBus N789,Paris - Marseille,3,NaN,NaN,73D700,FFFFFF,NaN,NaN
1120,FLIXBUS-eu,N702,FlixBus N702,Bruges - Paris - Brest,3,NaN,NaN,73D700,FFFFFF,NaN,NaN


Coordonnées de Paris

In [3]:
#FlixbusData

trajets = FlixbusData['route_long_name']

LATITUDE, LONGITUDE = 48.866667,  2.333333 # Latitude, longitude du centre de Paris

Récupération des villes, Fonctionnel mais récupère villes en doublons parfois à cause des différentes appellations

In [5]:
villes = []
villesReliees = {}

def ajout_tab(ajout, tableau):
  if not ajout in tableau:
    tableau.append(ajout)

regle1 = re.compile('(.*?)/(.*)')
regle2 = re.compile('(.*) \(.*')
regle3 = re.compile('(.*)\s*✈')
regle4 = re.compile('(.*) [aA]irport')


def nettoyage_nom_ville(ville):
  while True:
    if regle1.search(ville):
      res = regle1.match(ville)
      ville = res[2]
      ajout_tab(res[1], villes)
    else:
      break
  if regle2.search(ville):
    ville = regle2.match(ville)[1]
  if regle3.search(ville):
    ville = regle3.match(ville)[1]
  if regle4.search(ville):
    ville = regle4.match(ville)[1]
  ajout_tab(ville,villes)

prog = re.compile('(.*?)\s*-\s*(.*)')

def separer_villes(string,tableau):
  res = prog.match(string)
  nettoyage_nom_ville(res[1])
  if prog.search(res[2]):
    separer_villes(res[2],tableau)
  else:
    nettoyage_nom_ville(res[2])

for trajet in trajets:
  try:
    separer_villes(trajet,villes)
  except TypeError:
    print(trajet)
print(villes)


Hamburg – Berlin – Leipzig
Roskilde Festival event line
Modlin – Wroclaw – Ulm
['Prague', 'Pula', 'Chomutov', 'Most', 'Geneva', 'Milan', 'Bologna', 'Rome', 'Sorrento', 'Innsbruck', 'Munich', 'Hanover', 'Paris', 'Amsterdam', 'Utrecht', 'Brussels', 'Lyon', 'Clermont', 'Ferrand', 'Bordeaux', 'Hamburg', 'Bern', 'Koszalin', 'Szczecin', 'Berlin', 'Halle', 'Erfurt', 'Vilnius', 'Warsaw', 'Arad', 'Timisoara', 'Sofia', 'Turin', 'Ventimiglia', 'Vienna', 'Graz', 'Ancona', "L' Aquila", 'Rome FCO ', 'San Giovanni Rotondo', 'Nitra', 'Padova', 'Budapest', 'Ljubljana', 'Warszawa', 'Kraków', 'Kosice', 'Coimbra', 'Santarem', 'Lisbon', 'Katowice', 'Metz', 'Reims', 'Rennes', 'Zakopane', 'Kyiv', 'Kovel', 'Barcelona', 'Split', 'Dubrovnik', 'Wroclaw', 'Jelenia Gora', 'Swinoujscie', 'Rijeka', 'Bucharest', 'Salerno', 'Palermo', 'Calabria', 'Odesa', 'Chisinau', 'Luxembourg', 'Zagreb', 'Bratislava', 'Tours', 'Blois', 'Lviv', 'Nantes', 'Angers', 'Le Mans', 'Riga', 'Stuttgart', 'Wiesbaden', 'London', 'Povoa de Varz

Le code ci-dessous permet l'ajout des villes et des trajets directs en partant d'une ville précise (sans correspondance) (Toujours le problème des doublons qui vient des données pas très cleans )

In [6]:
villes = [] # Ensemble des villes
villesReliees = {} # Destinations possible sans correspondance en prenant comme clé une ville

def ajout_tab(ajout, tableau): # Fonction raccourci pour ajouter dans le tableau
  if not ajout in tableau:
    tableau.append(ajout)

# Règles basiques pour nettoyer le nom des villes
regle1 = re.compile('(.*?)/(.*)')
regle2 = re.compile('(.*) \(.*')
regle3 = re.compile('(.*)\s*✈')
regle4 = re.compile('(.*) [aA][ei]r.*')


def nettoyage_nom_ville(ville):
  while True:
    if regle1.search(ville):
      res = regle1.match(ville)
      ville = res[2]
      ajout_tab(res[1], villes)
    else:
      break
  if regle2.search(ville):
    ville = regle2.match(ville)[1]
  if regle3.search(ville):
    ville = regle3.match(ville)[1]
  if regle4.search(ville):
    ville = regle4.match(ville)[1]
  return ville

prog = re.compile('(.*?)\s*-\s*(.*)')


'''
Trajet de la forme
villeA - villeB - villeC
Ajout de villeB dans les destinations de villeA (J'ai pas compté aussi le trajet retour)

Si on retrouve le motif ça veut dire que c'était un trajet à plus de 2 villes et dans ce cas là on recommence en prenant villeA = villeB,
A la fin, ou directement, il n'y a plus que 2 villes sur le trajet villeB et villeC comme je fais rétrécir l'expression en enlevant villeA à chaque fois et donc on ajoute villeC aux destinations de villeB si pas déjà dedans
'''
def separer_villes(villeA,string,tableau):
  res = prog.match(string)
  villeB = nettoyage_nom_ville(res[1])
  ajout_tab(villeB, tableau)
  if (not villeA is None):
    if not villeA in villesReliees.keys():
      villesReliees[villeA] = []
    if(not villeB in villesReliees[villeA]):
      villesReliees[villeA].append(villeB)
  if prog.search(res[2]):
    separer_villes(villeB,res[2],tableau)
  else:
    villeC = nettoyage_nom_ville(res[2])
    if not villeB in villesReliees.keys():
      villesReliees[villeB] = []
    if not villeC in villesReliees[villeB]:
      villesReliees[villeB].append(villeC)
    ajout_tab(villeC, tableau)

for trajet in trajets:
  try:
    separer_villes(None,trajet,villes)
  except TypeError:
    print(trajet) # 3 Trajets ne sont pas traités je ne sais pas pourquoi
print(villes)
print(villesReliees)


Hamburg – Berlin – Leipzig
Roskilde Festival event line
Modlin – Wroclaw – Ulm
['Prague', 'Pula', 'Chomutov', 'Most', 'Geneva', 'Milan', 'Bologna', 'Rome', 'Sorrento', 'Innsbruck', 'Munich', 'Hanover', 'Paris', 'Amsterdam', 'Utrecht', 'Brussels', 'Lyon', 'Clermont', 'Ferrand', 'Bordeaux', 'Hamburg', 'Bern', 'Koszalin', 'Szczecin', 'Berlin', 'Halle', 'Erfurt', 'Vilnius', 'Warsaw', 'Arad', 'Timisoara', 'Sofia', 'Turin', 'Ventimiglia', 'Vienna', 'Graz', 'Ancona', "L' Aquila", 'Rome FCO ', 'San Giovanni Rotondo', 'Nitra', 'Padova', 'Budapest', 'Ljubljana', 'Warszawa', 'Kraków', 'Kosice', 'Coimbra', 'Santarem', 'Lisbon', 'Katowice', 'Metz', 'Reims', 'Rennes', 'Zakopane', 'Kyiv', 'Kovel', 'Barcelona', 'Split', 'Dubrovnik', 'Wroclaw', 'Jelenia Gora', 'Swinoujscie', 'Rijeka', 'Bucharest', 'Salerno', 'Palermo', 'Calabria', 'Odesa', 'Chisinau', 'Luxembourg', 'Zagreb', 'Bratislava', 'Tours', 'Blois', 'Lviv', 'Nantes', 'Angers', 'Le Mans', 'Riga', 'Stuttgart', 'Wiesbaden', 'London', 'Povoa de Varz

Récupération des coordonnées des villes acquéries pour les marqueurs de la carte, certaines ne font pas d'erreurs mais pourtant ne donnent pas de localisation, c'est pourquoi je retraite juste après

In [8]:
dicVilles = {}
listeVillesErreur = []

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='myapplication')
for ville in villes:
  try:
    getLoc = geolocator.geocode(ville)
    dicVilles[ville] = getLoc
  except:
    listeVillesErreur.append(ville)


Code ci-dessous utile pour pouvoir garder uniquement les villes qui avaient une localisation

In [9]:
dicVillesCorrectes = {}
for ville in dicVilles.keys():
  if not dicVilles[ville] is None:
    if dicVilles[ville].latitude > 30 and dicVilles[ville].longitude > -20 and dicVilles[ville].longitude < 40: # La ville n'est pas correcte si sa latitude / longitude n'est pas comprise en Europe donc pas la peine de l'ajouter
      dicVillesCorrectes[ville] = dicVilles[ville]

In [10]:
paris_map = folium.Map(location = [LATITUDE, LONGITUDE], tiles="cartodbpositron", zoom_start = 7)
color = ["red","green","yellow","blue","black"]


# Affiche tous les trajets existants mais surcharge
def afficherTousTrajets():
  for villeA in villesReliees.keys():
    if villeA in dicVillesCorrectes:
      for villeB in villesReliees[villeA]:
        if villeB in dicVillesCorrectes:
          points = []
          points.append(tuple([dicVillesCorrectes[villeA].latitude,dicVillesCorrectes[villeA].longitude]))
          points.append(tuple([dicVillesCorrectes[villeB].latitude,dicVillesCorrectes[villeB].longitude]))
          folium.PolyLine(points, color="red", weight=1, opacity=1).add_to(paris_map)

# Affiche tous les trajets partant d'une ville
def afficherTrajetVille(villeA):
  if villeA in dicVillesCorrectes.keys():
    for villeB in villesReliees[villeA]:
        if villeB in dicVillesCorrectes:
          points = []
          points.append(tuple([dicVillesCorrectes[villeA].latitude,dicVillesCorrectes[villeA].longitude]))
          points.append(tuple([dicVillesCorrectes[villeB].latitude,dicVillesCorrectes[villeB].longitude]))
          folium.PolyLine(points, color="red", weight=1, opacity=1).add_to(paris_map)

# Affiche tous les trajets partant d'une ville avec le nombre de correspondance maximal souhaité
def afficherTrajetCorrespondance(correspondances, nbCorrespondances):
  corr = []
  if nbCorrespondances == -1:
    return
  else:
    for correspondance in correspondances:
      if correspondance in dicVillesCorrectes.keys() and correspondance in villesReliees.keys():
        for correspondanceB in villesReliees[correspondance]:
          if correspondanceB in dicVillesCorrectes:
            if not correspondanceB in corr:
              corr.append(correspondanceB)
            points = []
            points.append(tuple([dicVillesCorrectes[correspondance].latitude,dicVillesCorrectes[correspondance].longitude]))
            points.append(tuple([dicVillesCorrectes[correspondanceB].latitude,dicVillesCorrectes[correspondanceB].longitude]))
            folium.PolyLine(points, color=color[nbCorrespondances], weight=1, opacity=1).add_to(paris_map)
    afficherTrajetCorrespondance(corr, nbCorrespondances-1)


def ajouterVilles():
  for ville in dicVillesCorrectes.values():
      folium.CircleMarker([ville.latitude, ville.longitude], radius = 2).add_to(paris_map)


ajouterVilles()
afficherTrajetCorrespondance(["Paris"],1) # Trajets de Paris avec 0 correspondance en vert et 1 correspondance en Rouge

paris_map